## Import Library

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pandas_ta as ta
import pspriceaction.price_action as pa
import pandas_ta as ta

## Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [4]:
%%time
data = pa.pattern_modeling(data)

CPU times: user 1min 20s, sys: 553 ms, total: 1min 21s
Wall time: 1min 20s


In [5]:
data["ma_line"] = data["Close"].rolling(20).mean()
data['above_ma'] = data.apply(lambda r: 1 if r['Close'] > r['ma_line'] else 0, axis=1)
data['below_ma'] = data.apply(lambda r: 1 if r['Close'] < r['ma_line'] else 0, axis=1)
data['total_above_ma'] = data['above_ma'].rolling(150).sum()
data['total_below_ma'] = data['below_ma'].rolling(150).sum()
data['total_above_ma_1w'] = data['above_ma'].rolling(250).sum()
data['total_below_ma_1w'] = data['below_ma'].rolling(250).sum()
data['trend_3d'] = data.apply(lambda r: 'switch' if r['total_above_ma'] == r['total_below_ma'] else ('up' if r['total_above_ma'] > r['total_below_ma'] else 'down'), axis=1)
data['trend_1w'] = data.apply(lambda r: 'switch' if r['total_above_ma_1w'] == r['total_below_ma_1w'] else ('up' if r['total_above_ma_1w'] > r['total_below_ma_1w'] else 'down'), axis=1)
data["RSI"] = ta.rsi(data["Close"], length=14)
data['upper_wick'] = data.apply(
    lambda r: r['High'] - max(r['Open'], r['Close']), axis=1)
data['tail'] = data.apply(
    lambda r: min(r['Open'], r['Close']) - r['Low'], axis=1)
data = data[['Open', 'High', 'Low', 'Close', 'trend_3d', 'trend_1w', "RSI", 'model', 'upper_wick', 'tail']]

In [6]:
data.dropna(inplace=True)

In [7]:
data.tail(10)

,Open,High,Low,Close,trend_3d,trend_1w,RSI,model,upper_wick,tail
Date,,,,,,,,,,
2024-08-02 13:50:00,1261.7,1263.3,1261.0,1262.4,down,down,47.943276,,0.9,0.7
2024-08-02 13:55:00,1262.2,1265.0,1262.0,1263.9,down,down,52.683699,,1.1,0.2
2024-08-02 14:00:00,1264.0,1270.1,1264.0,1269.4,down,down,65.197871,fair_value_rising_gap,0.7,0.0
2024-08-02 14:05:00,1269.2,1273.4,1269.2,1270.8,down,down,67.550480,fair_value_rising_gap,2.6,0.0
2024-08-02 14:10:00,1270.9,1271.6,1268.9,1271.2,down,down,68.211673,,0.4,2.0
2024-08-02 14:15:00,1271.4,1276.8,1270.7,1274.7,down,down,73.332053,,2.1,0.7
2024-08-02 14:20:00,1274.6,1277.5,1274.5,1276.8,down,down,75.846018,fair_value_rising_gap,0.7,0.1
2024-08-02 14:25:00,1276.4,1277.4,1274.0,1276.5,down,down,74.761752,,0.9,2.4
2024-08-02 14:30:00,1276.4,1276.4,1276.4,1276.4,down,down,74.380051,,0.0,0.0


## Strategy

In [8]:
def trade_simulation(prepared_data):
    _trades = pd.DataFrame(columns=("EntryTime", "EntryPrice", "ExitTime", "ExitPrice", "Type", "Profit", "Model", "RSI"))
    has_open_deal = False
    type_open_deal = ''
    # Stoploss at x0 pips
    sl_step = 1.5
    # Takeprofit at y0 pips(R/R = 1/3)
    tp_step = 4.5
    for i, row in prepared_data.iterrows():
        if 915 < 100*row.name.hour + row.name.minute < 1420:
            if has_open_deal:
                if type_open_deal == 'short':
                    # Stoploss
                    if row['High'] > entry_price + sl_step:
                        profit = -sl_step
                        exit_price = entry_price + sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit, entry_model, entry_rsi]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        # Takeprofit
                        if row['Low'] < entry_price - tp_step:
                            profit = tp_step
                            exit_price = entry_price - tp_step
                            exit_time = row.name
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit, entry_model, entry_rsi]
                            has_open_deal = False
                            type_open_deal = ''
                elif type_open_deal == 'long':
                    # Stoploss
                    if row['Low'] < entry_price - sl_step:
                        profit = -sl_step
                        exit_price = entry_price - sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit, entry_model, entry_rsi]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        # Takeprofit
                        if row['High'] > entry_price + tp_step:
                            profit = tp_step
                            exit_price = entry_price + tp_step
                            exit_time = row.name
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit, entry_model, entry_rsi]
                            has_open_deal = False
                            type_open_deal = ''
                        #
            if not has_open_deal:
                if row['signal'] == 'short':
                    # Open short deal
                    entry_price = row['Close']
                    entry_time = row.name
                    entry_rsi = row['RSI']
                    entry_model = row['model']
                    has_open_deal = True
                    type_open_deal = 'short'
                elif row['signal'] == 'long':
                    # Open short deal
                    entry_price = row['Close']
                    entry_time = row.name
                    entry_rsi = row['RSI']
                    entry_model = row['model']
                    has_open_deal = True
                    type_open_deal = 'long'
        else:
            if 100*row.name.hour + row.name.minute == 1425:
                if has_open_deal:
                    # close open deal at 2:25PM (dataframe)
                    if type_open_deal == 'short':
                        profit = entry_price - row['Close']
                        if profit < -sl_step:
                            profit = -sl_step
                        elif profit > tp_step:
                            profit = tp_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit, entry_model, entry_rsi]
                    elif type_open_deal == 'long':
                        profit = row['Close'] - entry_price
                        if profit < -sl_step:
                            profit = -sl_step
                        elif profit > tp_step:
                            profit = tp_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit, entry_model, entry_rsi]
    return _trades

In [9]:
def has_bullish_pattern(model):
    if "bullish" in model or "rising" in model:
        return True
    return False

def has_bearish_pattern(model):
    if "bearish" in model or "falling" in model:
        return True
    return False

def cal_signal_01(row):
    signal = ''
    # if row['trend_1w'] == 'up' and has_bullish_pattern(row['model']) and ((row['upper_wick'] < 1.61 and row['tail'] < 0.2) or (row['upper_wick'] < 0.21 and row['tail'] < 0.81)):
    # if has_bullish_pattern(row['model']) and row['upper_wick'] < 1.6 and row['tail'] < 2.5:
    # if row['model'] == 'bullish_neck':
    if row['trend_1w'] == 'up' and has_bullish_pattern(row['model']):
        signal = 'long'
    # elif row['trend_1w'] == 'down' and has_bearish_pattern(row['model']) and row['upper_wick'] < 0.3 and row['tail'] < 1.2:
    # elif has_bearish_pattern(row['model']) and row['upper_wick'] < 2.25 and row['tail'] < 2.1:
    # elif row['model'] == 'bearish_neck':
    elif row['trend_1w'] == 'down' and has_bearish_pattern(row['model']):
        signal = 'short'
    return signal
data['signal'] = data.apply(lambda r: cal_signal_01(r), axis=1)

In [10]:
data[data.signal != '']

,Open,High,Low,Close,trend_3d,trend_1w,RSI,model,upper_wick,tail,signal
Date,,,,,,,,,,,
2018-08-13 13:25:00,947.7,947.7,946.7,947.0,down,down,57.675929,fair_value_falling_gap,0.0,0.3,short
2018-08-14 09:05:00,954.8,955.0,954.3,955.0,down,down,79.741280,bearish_neck,0.0,0.5,short
2018-08-14 10:40:00,953.4,953.7,952.6,952.9,down,down,42.460630,fair_value_falling_gap,0.3,0.3,short
2018-08-15 10:10:00,960.3,960.3,959.9,960.1,down,down,54.318326,fair_value_falling_gap,0.0,0.2,short
2018-08-15 10:50:00,959.6,959.6,957.5,958.0,down,down,40.139232,fair_value_falling_gap,0.0,0.5,short
...,...,...,...,...,...,...,...,...,...,...,...
2024-07-31 10:35:00,1297.5,1298.0,1297.3,1297.8,up,up,70.302261,fair_value_rising_gap,0.2,0.2,long
2024-07-31 10:45:00,1298.8,1299.5,1298.7,1299.0,up,up,74.414827,fair_value_rising_gap,0.5,0.1,long
2024-07-31 10:50:00,1299.1,1300.9,1299.1,1300.3,up,up,78.125580,fair_value_rising_gap,0.6,0.0,long


In [11]:
trade_result = trade_simulation(data)

In [12]:
total_trades = len(trade_result)
profit = trade_result.Profit.sum()
trading_fee = total_trades * 0.3
win_rate = len(trade_result[trade_result.Profit > 0]) / total_trades
display_data = {
    'trades' : total_trades,
    'profit' : profit,
    'win rate' : win_rate,
    'trading_fee' : trading_fee,
    'profit_after_fee' : profit - trading_fee
}
display_data

{'trades': 3555,
 'profit': 979.4000000000008,
 'win rate': 0.320393811533052,
 'trading_fee': 1066.5,
 'profit_after_fee': -87.09999999999923}

In [13]:
trade_result

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit,Model,RSI
0,2018-08-13 13:25:00,947.0,2018-08-13 13:40:00,948.5,Short,-1.5,fair_value_falling_gap,57.675929
1,2018-08-14 10:40:00,952.9,2018-08-14 11:25:00,954.4,Short,-1.5,fair_value_falling_gap,42.460630
2,2018-08-15 10:10:00,960.1,2018-08-15 10:30:00,961.6,Short,-1.5,fair_value_falling_gap,54.318326
3,2018-08-15 10:50:00,958.0,2018-08-15 13:55:00,959.5,Short,-1.5,fair_value_falling_gap,40.139232
4,2018-08-15 14:05:00,956.2,2018-08-15 14:15:00,951.7,Short,4.5,fair_value_falling_gap,43.910870
...,...,...,...,...,...,...,...,...
3550,2024-07-30 14:10:00,1286.4,2024-07-30 14:15:00,1284.9,Long,-1.5,bullish_neck,42.464048
3551,2024-07-31 09:25:00,1295.6,2024-07-31 10:50:00,1300.1,Long,4.5,rising_three,74.416972
3552,2024-07-31 10:50:00,1300.3,2024-07-31 11:00:00,1298.8,Long,-1.5,fair_value_rising_gap,78.125580
3553,2024-07-31 11:30:00,1301.1,2024-07-31 13:20:00,1299.6,Long,-1.5,fair_value_rising_gap,73.541219


In [14]:
trade_result_loss = trade_result[trade_result.Profit < 0]

In [15]:
trade_result_win = trade_result[trade_result.Profit > 0]

In [16]:
grouped_l1 = trade_result_loss[['Model']].groupby([trade_result_loss.Model]).count()
grouped_l1['Model'].sort_values()

Model
bullish_gap, rising_three                1
bearish_gap, falling_n                   1
bearish_gap, falling_three               1
bullish_gap, rising_n                    1
rising_n                                13
falling_n                               25
bearish_gap, fair_value_falling_gap     28
bullish_gap, fair_value_rising_gap      32
rising_three                            48
bullish_gap                             50
falling_three                           54
bearish_gap                             55
bullish_separating_line                 95
bearish_neck                           314
bullish_neck                           318
fair_value_falling_gap                 665
fair_value_rising_gap                  707
Name: Model, dtype: int64

In [17]:
grouped_w1 = trade_result_win[['Model']].groupby([trade_result_win.Model]).count()
grouped_w1['Model'].sort_values()

Model
bearish_gap, falling_n                   1
bullish_gap, rising_n                    1
bullish_gap, rising_three                2
bearish_gap, falling_three               2
falling_n                               11
rising_n                                13
bearish_gap, fair_value_falling_gap     16
bullish_gap, fair_value_rising_gap      18
falling_three                           23
bullish_gap                             24
bearish_gap                             30
rising_three                            31
bullish_separating_line                 42
bullish_neck                           131
bearish_neck                           146
fair_value_falling_gap                 288
fair_value_rising_gap                  360
Name: Model, dtype: int64

In [18]:
this_month = trade_result[(trade_result.EntryTime > '2024-06-01 00:00:00') & (trade_result.EntryTime < '2024-07-30 15:00:00')]
this_month.Profit.sum()

25.5

In [19]:
this_year = trade_result[(trade_result.EntryTime > '2024-01-01 00:00:00') & (trade_result.EntryTime < '2024-06-30 15:00:00')]
this_year.Profit.sum()

63.59999999999991

In [20]:
from datetime import date
from datetime import timedelta
from datetime import datetime
last_expire_date = date.today() - timedelta(days=90)
last_expire_date = last_expire_date.strftime("%Y-%m-%d 00:00:00")
_last_3months = trade_result[trade_result.EntryTime > last_expire_date]
_last_3months.Profit.sum()

41.40000000000009

In [21]:
_last_3months

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit,Model,RSI
3423,2024-05-13 13:35:00,1272.5,2024-05-13 13:40:00,1271.0,Long,-1.5,fair_value_rising_gap,64.260375
3424,2024-05-14 09:25:00,1275.1,2024-05-14 10:45:00,1273.6,Long,-1.5,bullish_separating_line,64.106507
3425,2024-05-14 14:00:00,1273.1,2024-05-14 14:25:00,1276.3,Long,3.2,bullish_neck,49.874594
3426,2024-05-15 13:05:00,1287.6,2024-05-15 13:45:00,1292.1,Long,4.5,fair_value_rising_gap,78.262246
3427,2024-05-15 13:45:00,1291.9,2024-05-15 13:50:00,1290.4,Long,-1.5,fair_value_rising_gap,85.102907
...,...,...,...,...,...,...,...,...
3550,2024-07-30 14:10:00,1286.4,2024-07-30 14:15:00,1284.9,Long,-1.5,bullish_neck,42.464048
3551,2024-07-31 09:25:00,1295.6,2024-07-31 10:50:00,1300.1,Long,4.5,rising_three,74.416972
3552,2024-07-31 10:50:00,1300.3,2024-07-31 11:00:00,1298.8,Long,-1.5,fair_value_rising_gap,78.125580
3553,2024-07-31 11:30:00,1301.1,2024-07-31 13:20:00,1299.6,Long,-1.5,fair_value_rising_gap,73.541219


In [22]:
trade_result['EntryTime'] = trade_result['EntryTime']
k = trade_result['Profit'].groupby([trade_result.EntryTime.dt.year]).sum()
k

EntryTime
2018     56.0
2019     77.9
2020    296.1
2021    212.9
2022    121.1
2023    129.3
2024     86.1
Name: Profit, dtype: float64

In [23]:
data[(data.index > '2024-07-18 08:50:00') & (data.index < '2024-07-18 14:50:00')]

,Open,High,Low,Close,trend_3d,trend_1w,RSI,model,upper_wick,tail,signal
Date,,,,,,,,,,,
2024-07-18 09:00:00,1303.1,1304.3,1301.7,1303.9,up,down,45.162043,,0.4,1.4,
2024-07-18 09:05:00,1304.0,1304.0,1302.8,1303.0,up,down,43.897793,,0.0,0.2,
2024-07-18 09:10:00,1303.0,1303.6,1303.0,1303.5,up,down,44.821934,,0.1,0.0,
2024-07-18 09:15:00,1303.5,1305.0,1302.9,1303.1,up,down,44.194738,,1.5,0.2,
2024-07-18 09:20:00,1303.1,1303.3,1300.9,1301.2,up,down,41.242597,,0.2,0.3,
2024-07-18 09:25:00,1301.3,1302.4,1300.9,1302.4,up,down,43.796150,,0.0,0.4,
2024-07-18 09:30:00,1302.1,1303.4,1302.1,1303.4,switch,down,45.905922,,0.0,0.0,
2024-07-18 09:35:00,1303.3,1304.0,1302.9,1303.0,down,down,45.175428,,0.7,0.1,
2024-07-18 09:40:00,1303.0,1303.4,1302.6,1303.0,down,down,45.175428,,0.4,0.4,
